# Diabetic Retinopathy - Kaggle

In [9]:
# importing pachages

import os

import pandas as pd
import numpy as np
import time

import torch

import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import confusion_matrix

In [10]:
# defining dataset object

class TestData(Dataset):
    '''dataset object'''
    
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('aptos2019-blindness-detection/train_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = self.transform(image)
        diagnosis = self.data.diagnosis[idx]
        return image,diagnosis


In [13]:
# defining transformations to do in the images when loaded (resizing to 264X264 and tranforming into tensor)

transformations = transforms.Compose([   
    transforms.Resize((264, 264)),     
    transforms.ToTensor()])

In [15]:
# creating an instance of the dataset object

dataset = TestData(csv_file='aptos2019-blindness-detection/train.csv',
                                      transform=transformations)

## Train Validation Split

In [16]:
validation_split = .2   # validation with 20% of the base
shuffle_dataset = True
random_seed= 8


# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [17]:
# seting batch size
batch_size = 25

# creating train and validation loaders
train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

## Defining CNN Structure

In [51]:
class DR_net(nn.Module):
    
    def __init__(self):
        
        super(DR_net,self).__init__()
        
        self.conv1 = nn.Conv2d(3,20,3)
        self.conv2 = nn.Conv2d(20,30,5)
        self.conv3 = nn.Conv2d(30,40,3)
        self.conv4 = nn.Conv2d(40,50,5)        
        
        self.fc1 = nn.Linear(50*13*13,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        self.out = nn.Linear(10,5)
        
    def forward(self, x):
        
        # convolutional layers
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = F.max_pool2d(F.relu(self.conv3(x)),2)
        x = F.max_pool2d(F.relu(self.conv4(x)),2)
        
        # Dense layers
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        # output layer
        x = self.out(x)
        #x = F.softmax(x, dim=1) #not required here since the loss function (cross entropy) uses sofmax implilcitly
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

## Instatiating the network and setting hyperparameters for training

In [52]:
# instantiation
CNN = DR_net()

# hyperparameters
optimizer = optim.Adam(CNN.parameters(), lr=0.005)
n_epochs = 2
criterion = F.cross_entropy  # nn.MSELoss


## Training

In [50]:
start = time.time()
for epoch in range(n_epochs):  # loop over the dataset multiple times
    
    print("                 --- Epoch: {} ---".format(epoch + 1))
    running_loss = 0.0
    cor_pred = np.array(0)
    running_n = np.array(0)
    
    for i, batch in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = CNN(inputs)
        loss = criterion(outputs, labels)
        
        cor_pred = cor_pred + outputs.argmax(dim=1).eq(labels).sum().numpy()
        running_n = running_n + len(labels)
        perc = cor_pred/running_n
        
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] train loss: %.3f train accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10, perc))
            running_loss = 0.0
            
print('Finished Training')

end = time.time()
runingtime = (end - start)
print('Training time : %5d mins' % (runingtime/60))

correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        images, labels = data
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the ',str(len(valid_sampler)),' validation images: %d %%' % (100 * correct / total))

                 --- Epoch: 1 ---


RuntimeError: size mismatch, m1: [25 x 8450], m2: [6050 x 120] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:961

## Confusion Matrix

In [22]:
def quadratic_kappa(actuals, preds, N=5):
    """This function calculates the Quadratic Kappa Metric used for Evaluation in the PetFinder competition
    at Kaggle. It returns the Quadratic Weighted Kappa metric score between the actual and the predicted values 
    of adoption rating."""
    w = np.zeros((N,N))
    O = confusion_matrix(actuals, preds)
    for i in range(len(w)): 
        for j in range(len(w)):
            w[i][j] = float(((i-j)**2)/(N-1)**2)
    
    act_hist=np.zeros([N])
    for item in actuals: 
        act_hist[item]+=1
    
    pred_hist=np.zeros([N])
    for item in preds: 
        pred_hist[item]+=1
                         
    E = np.outer(act_hist, pred_hist);
    E = E/E.sum();
    O = O/O.sum();
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*O[i][j]
            den+=w[i][j]*E[i][j]
    return (1 - (num/den))

In [23]:
@torch.no_grad()
def DS_preds(model,loader):
    ds_preds = torch.tensor([])
    for batch in loader:
        images,labels = batch
        preds = model(images)
        ds_preds = torch.cat(
                (ds_preds,preds),dim = 0)

    return ds_preds

In [58]:
start = time.time()
for epoch in range(n_epochs):  # loop over the dataset multiple times
    
    print("                 --- Epoch: {} ---".format(epoch + 1))
    running_loss = 0.0
    cor_pred = np.array(0)
    running_n = np.array(0)
    
    for i, batch in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = CNN(inputs)
        loss = criterion(outputs, labels)
        
        cor_pred = cor_pred + outputs.argmax(dim=1).eq(labels).sum().numpy()
        running_n = running_n + len(labels)
        perc = cor_pred/running_n
        
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] train loss: %.3f train accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10, perc))
            running_loss = 0.0
    
    
    valid_preds = DS_preds(CNN, validation_loader)
    valid_preds = pd.Series(valid_preds.argmax(dim=1).numpy())
    valid_labels = dataset.data.diagnosis[val_indices].reset_index(drop = True) 
    valid_kaṕpa = quadratic_kappa(valid_labels, valid_preds, N=5)
    print('The Quadratic Kappa Metric in the ',str(len(valid_sampler)),' validation images: %d %%' % (valid_kappa))


print('Finished Training')

end = time.time()
runingtime = (end - start)
print('Training time : %5d mins' % (runingtime/60))



                 --- Epoch: 1 ---
[1,    10] train loss: 0.861 train accuracy: 0.704
[1,    20] train loss: 1.031 train accuracy: 0.678
[1,    30] train loss: 0.858 train accuracy: 0.691
[1,    40] train loss: 0.766 train accuracy: 0.701
[1,    50] train loss: 0.840 train accuracy: 0.706
[1,    60] train loss: 0.874 train accuracy: 0.701
[1,    70] train loss: 0.891 train accuracy: 0.699
[1,    80] train loss: 0.812 train accuracy: 0.699
[1,    90] train loss: 0.906 train accuracy: 0.695
[1,   100] train loss: 0.792 train accuracy: 0.697
[1,   110] train loss: 0.913 train accuracy: 0.694


NameError: name 'valid_kappa' is not defined

In [59]:
data_preds = DS_preds(CNN, train_loader)

In [60]:
data_preds = data_preds.argmax(dim=1)

In [61]:
data_preds = pd.Series(data_preds.numpy())# = data_preds.argmax(dim=1)

data_labels = dataset.data.diagnosis[train_indices].reset_index(drop = True) 

In [62]:
quadratic_kappa(data_labels, data_preds, N=5)

0.016338350948726266

In [65]:
confusion_matrix(data_labels,data_preds)

array([[834,   0, 599,   0,   0],
       [160,   0, 125,   0,   0],
       [462,   0, 358,   0,   0],
       [ 95,   0,  58,   0,   0],
       [128,   0, 111,   0,   0]])

In [66]:
(834+358)/2930

0.4068259385665529